```
Author: Ehsan Kamalinejad (EK)
Created: 2023-02-25
```

# PPO Training

This notebook is a basic implementation of reinforcement learning (RL) training through proximal policy optimization (PPO). PPO is the default RL training for many problems at OpenAI and has great performance while very flexible. Here, the focus is education and simplicity.

Here, we will learn how to train a RL agent from scratch through PPO. This will be useful when we do (reinforcement learning with human feedback) RLHF to fine-tune language models in future lectures.

Pre-requisites:
- Intermediate level familiarity with Python and PyTorch.
- Intermediate level familiarity with general concepts in machine learning (ML) and gradient based optimization.
- Basic familiarity with concepts in RL such as environments, rewards, agents, etc.

Dependencies:
```
pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
pip install moviepy omegaconf matplotlib
pip install gym==0.26.2
pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests
pip install gym[classic_control] gym[atari] gym[accept-rom-license] gym[other]
```

References:
- [EK's Video Lecture](https://www.youtube.com/watch?v=3uvnoVjM8nY) This is the lecture where we did a deep dive into the theory of PPO.
- [OpenAI PPO Repo](https://github.com/openai/baselines/blob/master/baselines/ppo2/runner.py) This is helpful as a reference for further implementations.
- [PPO Paper](https://arxiv.org/abs/1707.06347) This is the original paper that introduced PPO.
- [Sergey Levine UC Berkley CS285](http://rail.eecs.berkeley.edu/deeprlcourse/) This is a complete course in RL.
- [Pieter Abbeel mini-course](https://www.youtube.com/watch?v=2GwBez0D20A&list=PLwRJQ4m4UJjNymuBM9RdmB3Z9N5-0IlY0) This is a mini-course focusing on TRPO, PPO, DDPG and model free RL.
- [OpenAI Documentation on RL](https://spinningup.openai.com/en/latest/index.html) THis is OpenAI documentation on RL and parts of our code was borrowed from here.
- [labml.ai](https://nn.labml.ai/) This repo contains popular papers with their annotated PyTorch implementations.
- [cleanrl](https://github.com/vwxyzjn/cleanrl) This repo has clean implementations of RL algorithms and parts of our code was borrowed from here.

In [1]:
# !pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
!pip install moviepy omegaconf matplotlib
!pip install gym==0.26.2
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests
!pip install gym[classic_control] gym[atari] gym[accept-rom-license] gym[other]

  Obtaining dependency information for omegaconf from https://files.pythonhosted.org/packages/e3/94/1843518e420fa3ed6919835845df698c7e27e183cb997394e4a670973a65/omegaconf-2.3.0-py3-none-any.whl.metadata
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Preparing metadata (setup.py) ... done
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=dee7a7ea43993b6873a76982ff4e38d940859e768a2f5dc739d10be4b80323a4
  Stored in directory: /Users/cohlem/Library/Caches/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for gym_notices>=0.0.4 

  Obtaining dependency information for importlib-metadata~=4.13 from https://files.pythonhosted.org/packages/d0/98/c277899f5aa21f6e6946e1c83f2af650cbfee982763ffb91db07ff7d3a13/importlib_metadata-4.13.0-py3-none-any.whl.metadata
  Created wheel for stable_baselines3: filename=stable_baselines3-2.0.0a0-py3-none-any.whl size=174675 sha256=67d7e9f421066cf3a81cbb1e571345da13ae11bfb581cf1a862984484c478aa2
  Stored in directory: /private/var/folders/pv/2c84c3jx48v8485vx92jq6zc0000gn/T/pip-ephem-wheel-cache-8icna2yc/wheels/cc/3f/36/c2107a7756801b30e445eb0eea4eccce687ca61229763e2a3f
Successfully built stable_baselines3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 8.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446709 sha256=81134d21c8b4ff11e2975b101c4da4e52086564918ba89b3bda3ffa635dd1d42
  Stored in directory: /Users/cohlem/Library/Caches/pip/wheels/bc/fc/c6/8aa657c0d2089982f2dabd110efc68c61eb49831fdb7397351
Successfully built AutoROM.accept-rom-license


In [2]:
import time
import random
import numpy as np
import matplotlib.pylab as plt
plt.style.use('dark_background')
from tqdm.notebook import tqdm
from omegaconf import DictConfig

import gymnasium as gym

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


from IPython.display import Video

## Setup

In [3]:
seed = 2023
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
configs = {
    # experiment arguments
    "exp_name": "cartpole",
    "gym_id": "CartPole-v1", # the id of from OpenAI gym
    # training arguments
    "learning_rate": 1e-3, # the learning rate of the optimizer
    "total_timesteps": 1000000, # total timesteps of the training
    "max_grad_norm": 0.5, # the maximum norm allowed for the gradient
    # PPO parameters
    "num_trajcts": 32, # N
    "max_trajects_length": 64, # T
    "gamma": 0.99, # gamma
    "gae_lambda":0.95, # lambda for the generalized advantage estimation
    "num_minibatches": 2, # number of mibibatches used in each gradient
    "update_epochs": 2, # number of full rollout storage creations
    "clip_epsilon": 0.2, # the surrogate clipping coefficient
    "ent_coef": 0.01, # entroy coefficient controlling the exploration factor C2
    "vf_coef": 0.5, # value function controlling value estimation importance C1
    # visualization and print parameters
    "num_returns_to_average": 3, # how many episodes to use for printing average return
    "num_episodes_to_average": 23, # how many episodes to use for smoothing of the return diagram
    }

# batch_size is the size of the flatten sequences when trajcts are flatten
configs['batch_size'] = int(configs['num_trajcts'] * configs['max_trajects_length'])
# number of samples used in each gradient
configs['minibatch_size'] = int(configs['batch_size'] // configs['num_minibatches'])

configs = DictConfig(configs)

run_name = f"{configs.gym_id}__{configs.exp_name}__{seed}__{int(time.time())}"

In [53]:
12

12

In [55]:
sample = torch.zeros((2,4))

In [60]:
sample[0, 1:4] = 1

In [61]:
sample

tensor([[0., 1., 1., 1.],
        [0., 0., 0., 0.]])

## Env

`envs` is a set of parallel environments each holding a random initiali `state` and accepts an `action` to change and return its new state.

In [5]:
# create an env with random state
def make_env_func(gym_id, seed, idx, run_name, capture_video=False):
    def env_fun():
        env = gym.make(gym_id, render_mode="rgb_array")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            # initiate the video capture if not already initiated
            if idx == 0:
                # wrapper to create the video of the performance
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return env_fun

In [6]:
# create N envs
envs = []
for i in range(configs.num_trajcts):
    envs.append( make_env_func(configs.gym_id, seed + i, i, run_name) )
envs = gym.vector.SyncVectorEnv(envs)
envs

SyncVectorEnv(num_envs=32)

## Model

A simple fully connected model that gets a state and has two methods:
- `agent.value_func(state)` gets a state and returns the estimated expected total future rewards from that state $V_{\theta}(s)$.
- `agent.policy(state)` gets a state and returns next `action`, `log_prob` of actions, the `entropy` and `value`.

In [7]:
class FCBlock(nn.Module):
    """A generic fully connected residual block with good setup"""
    def __init__(self, embd_dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.LayerNorm(embd_dim),
            nn.GELU(),
            nn.Linear(embd_dim, 4*embd_dim),
            nn.GELU(),
            nn.Linear(4*embd_dim, embd_dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return x + self.block(x)


class Agent(nn.Module):
    """an agent that creates actions and estimates values"""
    def __init__(self, env_observation_dim, action_space_dim, embd_dim=64, num_blocks=2):
        super().__init__()
        self.embedding_layer = nn.Linear(env_observation_dim, embd_dim)
        self.shared_layers = nn.Sequential(*[FCBlock(embd_dim=embd_dim) for _ in range(num_blocks)])
        self.value_head = nn.Linear(embd_dim, 1)
        self.policy_head = nn.Linear(embd_dim, action_space_dim)
        # orthogonal initialization with a hi entropy for more exploration at the start
        torch.nn.init.orthogonal_(self.policy_head.weight, 0.01)

    def value_func(self, state):
        hidden = self.shared_layers(self.embedding_layer(state))
        value = self.value_head(hidden)
        return value

    def policy(self, state, action=None):
        hidden = self.shared_layers(self.embedding_layer(state))
        logits = self.policy_head(hidden)
        # PyTorch categorical class helpful for sampling and probability calculations
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.value_head(hidden)


In [8]:
import torch
from torch.distributions.categorical import Categorical

# Define a categorical distribution with specified probabilities
probs = torch.tensor([0.1, 0.2, 0.3, 0.4])  # Probabilities for 4 outcomes
dist = Categorical(probs)

# Sample from the distribution
sample = dist.sample()
print("Sampled outcome:", sample.item())

# Calculate the log probability of an outcome
log_prob = dist.log_prob(torch.tensor(2))  # Log probability of outcome 2
print("Log probability of outcome 2:", log_prob.item())

# Calculate the entropy of the distribution
entropy = dist.entropy()
print("Entropy of the distribution:", entropy.item())


Sampled outcome: 3
Log probability of outcome 2: -1.2039728164672852
Entropy of the distribution: 1.2798542976379395


### Generalized Advantage Estimation

In [9]:
def gae(
    cur_observation,  # the current state when advantages will be calculated
    rewards,          # rewards collected from trajectories of shape [num_trajcts, max_trajects_length]
    dones,            # binary marker of end of trajectories of shape [num_trajcts, max_trajects_length]
    values            # value estimates collected over trajectories of shape [num_trajcts, max_trajects_length]
):
    """
    Generalized Advantage Estimation (gae) estimating advantage of a particular trajecotry
    vs the expected return starting from a state
    """
    advantages = torch.zeros((configs.num_trajcts, configs.max_trajects_length))
    last_advantage = 0

    # the value after the last step
    with torch.no_grad():
        last_value = agent.value_func(cur_observation).reshape(1, -1)
#         last_value = agent.value_func(cur_observation)


    # reverse recursive to calculate advantages based on the delta formula
    for t in reversed(range(configs.max_trajects_length)):
        # mask if episode completed after step t
        

        mask = 1.0 - dones[:, t]
        last_value = last_value * mask
        
        last_advantage = last_advantage * mask
        delta = rewards[:, t] + configs.gamma * last_value - values[:, t]
        last_advantage = delta + configs.gamma * configs.gae_lambda * last_advantage

        
        advantages[:, t] = last_advantage
        last_value = values[:, t]

    advantages = advantages.to(device)
    returns = advantages + values

    return advantages, returns

### Creating Rollout Storage

In [10]:
def create_rollout(
    envs,            # parallel envs creating trajectories
    cur_observation, # starting observation of shape [num_trajcts, observation_dim]
    cur_done,        # current termination status of shape [num_trajcts,]
    all_returns      # a list to track returns
):
    """
    rollout phase: create parallel trajectories and store them in the rollout storage
    """

    # cache empty tensors to store the rollouts
    observations = torch.zeros((configs.num_trajcts, configs.max_trajects_length) +
                               envs.single_observation_space.shape).to(device)
    actions = torch.zeros((configs.num_trajcts, configs.max_trajects_length) +
                          envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)
    rewards = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)
    dones = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)
    values = torch.zeros((configs.num_trajcts, configs.max_trajects_length)).to(device)

    for t in range(configs.max_trajects_length):
        observations[:,t] = cur_observation
        dones[:,t] = cur_done

        # give observation to the model and collect action, logprobs of actions, entropy and value
        with torch.no_grad():
            action, logprob, entropy, value = agent.policy(cur_observation)
        values[:,t] = value.flatten()
        actions[:,t] = action
        logprobs[:,t] = logprob

        # apply the action to the env and collect observation and reward
        cur_observation, reward, cur_done, _, info = envs.step(action.cpu().numpy())
        rewards[:,t] = torch.tensor(reward).to(device).view(-1)
        cur_observation = torch.Tensor(cur_observation).to(device)
        cur_done = torch.Tensor(cur_done).to(device)

        # if an episode ended store its total reward for progress report
#         if info:
#             for item in info['final_info']:
#                 if item and "episode" in item.keys():
#                     all_returns.append(item['episode']['r'])
#                     break

    # create the rollout storage
    rollout = {
        'cur_observation': cur_observation,
        'cur_done': cur_done,
        'observations': observations,
        'actions': actions,
        'logprobs': logprobs,
        'values': values,
        'dones': dones,
        'rewards': rewards
    }

    return rollout

In [11]:
class Storage(Dataset):
    def __init__(self, rollout, advantages, returns, envs):
        # fill in the storage and flatten the parallel trajectories
        self.observations = rollout['observations'].reshape((-1,) + envs.single_observation_space.shape)
        self.logprobs = rollout['logprobs'].reshape(-1)
        self.actions = rollout['actions'].reshape((-1,) + envs.single_action_space.shape).long()
        self.advantages = advantages.reshape(-1)
        self.returns = returns.reshape(-1)

    def __getitem__(self, ix: int):
        item = [
            self.observations[ix],
            self.logprobs[ix],
            self.actions[ix],
            self.advantages[ix],
            self.returns[ix]
        ]
        return item

    def __len__(self) -> int:
        return len(self.observations)

### Loss Functions

In [12]:
def loss_clip(
    mb_oldlogporb,     # old logprob of mini batch actions collected during the rollout
    mb_newlogprob,     # new logprob of mini batch actions created by the new policy
    mb_advantages      # mini batch of advantages collected during the the rollout
):
    """
    policy loss with clipping to control gradients
    """
    ratio = torch.exp(mb_newlogprob - mb_oldlogporb)
    policy_loss = -mb_advantages * ratio
    # clipped policy gradient loss enforces closeness
    clipped_loss = -mb_advantages * torch.clamp(ratio, 1 - configs.clip_epsilon, 1 + configs.clip_epsilon)
    pessimistic_loss = torch.max(policy_loss, clipped_loss).mean()
    return pessimistic_loss


def loss_vf(
    mb_oldreturns,  # mini batch of old returns collected during the rollout
    mb_newvalues    # minibach of values calculated by the new value function
):
    """
    enforcing the value function to give more accurate estimates of returns
    """
    mb_newvalues = mb_newvalues.view(-1)
    loss = 0.5 * ((mb_newvalues - mb_oldreturns) ** 2).mean()
    return loss

## Training

In [13]:
agent = Agent(
    env_observation_dim=envs.single_observation_space.shape[0],
    action_space_dim=envs.single_action_space.n
).to(device)

optimizer = optim.Adam(agent.parameters(), lr=configs.learning_rate)

In [98]:
all_returns = []

# initialize the game
cur_observation = torch.Tensor(envs.reset()[0]).to(device)
cur_done = torch.zeros(configs.num_trajcts).to(device)

In [99]:
configs.minibatch_size

1024

In [100]:
# track returns
all_returns = []

# initialize the game
cur_observation = torch.Tensor(envs.reset()[0]).to(device)
cur_done = torch.zeros(configs.num_trajcts).to(device)

# progress bar
num_updates = configs.total_timesteps // configs.batch_size
progress_bar = tqdm(total=num_updates)

for update in range(1, num_updates + 1):


    ##############################################
    # Phase 1: rollout creation

    # parallel envs creating trajectories
    rollout = create_rollout(envs, cur_observation, cur_done, all_returns)

    cur_done = rollout['cur_done']
    cur_observation = rollout['cur_observation']
    rewards = rollout['rewards']
    dones = rollout['dones']
    values = rollout['values']

    # calculating advantages
    advantages, returns = gae(cur_observation, rewards, dones, values)

    # a dataset containing the rollouts
    dataset = Storage(rollout, advantages, returns, envs)

    print(len(dataset))
    # a standard dataloader made out of current storage
    trainloader = DataLoader(dataset, batch_size=configs.minibatch_size, shuffle=True)


    ##############################################
    # Phase 2: model update

    # linearly shrink the lr from the initial lr to zero
    frac = 1.0 - (update - 1.0) / num_updates
    optimizer.param_groups[0]["lr"] = frac * configs.learning_rate

    # training loop
    for epoch in range(configs.update_epochs):
        for batch in trainloader:
            mb_observations, mb_logprobs, mb_actions, mb_advantages, mb_returns = batch

            # we calculate the distribution of actions through the updated model revisiting the old trajectories
            _, mb_newlogprob, mb_entropy, mb_newvalues = agent.policy(mb_observations, mb_actions)

            policy_loss = loss_clip(mb_logprobs, mb_newlogprob, mb_advantages)

            value_loss = loss_vf(mb_returns, mb_newvalues)

            # average entory of the action space
            entropy_loss = mb_entropy.mean()

            # full weighted loss
            loss = policy_loss - configs.ent_coef * entropy_loss + configs.vf_coef * value_loss

            print(loss)
            optimizer.zero_grad()
            loss.backward()

            # extra clipping of the gradients to avoid overshoots
            nn.utils.clip_grad_norm_(agent.parameters(), configs.max_grad_norm)
            optimizer.step()

    # progress bar
    if len(all_returns) > configs.num_returns_to_average:
        progress_bar.set_description(f"episode return: {np.mean(all_returns[-configs.num_returns_to_average:]):.2f}")
        progress_bar.refresh()
        progress_bar.update()
envs.close()

  0%|          | 0/488 [00:00<?, ?it/s]

2048
tensor(12.5200, grad_fn=<AddBackward0>)
tensor(11.2932, grad_fn=<AddBackward0>)
tensor(9.5838, grad_fn=<AddBackward0>)
tensor(8.5303, grad_fn=<AddBackward0>)
2048
tensor(10.4960, grad_fn=<AddBackward0>)
tensor(8.8961, grad_fn=<AddBackward0>)
tensor(7.8095, grad_fn=<AddBackward0>)
tensor(5.8538, grad_fn=<AddBackward0>)
2048
tensor(7.6547, grad_fn=<AddBackward0>)
tensor(6.2848, grad_fn=<AddBackward0>)
tensor(4.8548, grad_fn=<AddBackward0>)
tensor(3.2112, grad_fn=<AddBackward0>)
2048
tensor(8.4795, grad_fn=<AddBackward0>)
tensor(6.2684, grad_fn=<AddBackward0>)
tensor(4.6960, grad_fn=<AddBackward0>)
tensor(3.9718, grad_fn=<AddBackward0>)
2048
tensor(10.1310, grad_fn=<AddBackward0>)
tensor(8.5069, grad_fn=<AddBackward0>)
tensor(7.3593, grad_fn=<AddBackward0>)
tensor(6.1121, grad_fn=<AddBackward0>)
2048
tensor(12.7728, grad_fn=<AddBackward0>)
tensor(12.6545, grad_fn=<AddBackward0>)
tensor(13.1728, grad_fn=<AddBackward0>)
tensor(13.1289, grad_fn=<AddBackward0>)
2048
tensor(16.9630, grad_

tensor(21.3628, grad_fn=<AddBackward0>)
tensor(22.6316, grad_fn=<AddBackward0>)
tensor(19.8418, grad_fn=<AddBackward0>)
2048
tensor(22.0911, grad_fn=<AddBackward0>)
tensor(24.0254, grad_fn=<AddBackward0>)
tensor(22.8915, grad_fn=<AddBackward0>)
tensor(23.1287, grad_fn=<AddBackward0>)
2048
tensor(58.3377, grad_fn=<AddBackward0>)
tensor(71.5483, grad_fn=<AddBackward0>)
tensor(66.2170, grad_fn=<AddBackward0>)
tensor(60.9500, grad_fn=<AddBackward0>)
2048
tensor(23.4455, grad_fn=<AddBackward0>)
tensor(24.4350, grad_fn=<AddBackward0>)
tensor(21.1959, grad_fn=<AddBackward0>)
tensor(25.1333, grad_fn=<AddBackward0>)
2048
tensor(26.7439, grad_fn=<AddBackward0>)
tensor(26.3652, grad_fn=<AddBackward0>)
tensor(25.0380, grad_fn=<AddBackward0>)
tensor(27.3200, grad_fn=<AddBackward0>)
2048
tensor(20.2687, grad_fn=<AddBackward0>)
tensor(19.8037, grad_fn=<AddBackward0>)
tensor(18.2750, grad_fn=<AddBackward0>)
tensor(22.1706, grad_fn=<AddBackward0>)
2048
tensor(16.5483, grad_fn=<AddBackward0>)
tensor(16.

tensor(20.2355, grad_fn=<AddBackward0>)
tensor(24.4227, grad_fn=<AddBackward0>)
tensor(20.6883, grad_fn=<AddBackward0>)
2048
tensor(22.6260, grad_fn=<AddBackward0>)
tensor(19.9253, grad_fn=<AddBackward0>)
tensor(22.2208, grad_fn=<AddBackward0>)
tensor(19.7763, grad_fn=<AddBackward0>)
2048
tensor(8.2520, grad_fn=<AddBackward0>)
tensor(7.2097, grad_fn=<AddBackward0>)
tensor(7.1564, grad_fn=<AddBackward0>)
tensor(9.2074, grad_fn=<AddBackward0>)
2048
tensor(21.7018, grad_fn=<AddBackward0>)
tensor(22.4772, grad_fn=<AddBackward0>)
tensor(19.5793, grad_fn=<AddBackward0>)
tensor(25.5296, grad_fn=<AddBackward0>)
2048
tensor(21.5828, grad_fn=<AddBackward0>)
tensor(21.0041, grad_fn=<AddBackward0>)
tensor(19.0576, grad_fn=<AddBackward0>)
tensor(23.6924, grad_fn=<AddBackward0>)
2048
tensor(9.5945, grad_fn=<AddBackward0>)
tensor(9.6387, grad_fn=<AddBackward0>)
tensor(9.3784, grad_fn=<AddBackward0>)
tensor(8.7270, grad_fn=<AddBackward0>)
2048
tensor(24.3616, grad_fn=<AddBackward0>)
tensor(21.1731, gr

tensor(14.0100, grad_fn=<AddBackward0>)
tensor(15.7087, grad_fn=<AddBackward0>)
tensor(15.9030, grad_fn=<AddBackward0>)
tensor(14.0642, grad_fn=<AddBackward0>)
2048
tensor(14.7521, grad_fn=<AddBackward0>)
tensor(13.8309, grad_fn=<AddBackward0>)
tensor(15.8963, grad_fn=<AddBackward0>)
tensor(10.9333, grad_fn=<AddBackward0>)
2048
tensor(13.7706, grad_fn=<AddBackward0>)
tensor(10.6997, grad_fn=<AddBackward0>)
tensor(12.5791, grad_fn=<AddBackward0>)
tensor(12.1680, grad_fn=<AddBackward0>)
2048
tensor(13.8878, grad_fn=<AddBackward0>)
tensor(26.6002, grad_fn=<AddBackward0>)
tensor(17.0177, grad_fn=<AddBackward0>)
tensor(20.6987, grad_fn=<AddBackward0>)
2048
tensor(25.7322, grad_fn=<AddBackward0>)
tensor(25.9837, grad_fn=<AddBackward0>)
tensor(27.2851, grad_fn=<AddBackward0>)
tensor(23.1378, grad_fn=<AddBackward0>)
2048
tensor(13.1275, grad_fn=<AddBackward0>)
tensor(15.5818, grad_fn=<AddBackward0>)
tensor(15.1690, grad_fn=<AddBackward0>)
tensor(13.5136, grad_fn=<AddBackward0>)
2048
tensor(4.5

...

tensor(-0.0195, grad_fn=<AddBackward0>)
tensor(-0.0386, grad_fn=<AddBackward0>)
tensor(-0.0094, grad_fn=<AddBackward0>)
2048
tensor(0.2359, grad_fn=<AddBackward0>)
tensor(0.3033, grad_fn=<AddBackward0>)
tensor(0.2817, grad_fn=<AddBackward0>)
tensor(0.2603, grad_fn=<AddBackward0>)
2048
tensor(0.0829, grad_fn=<AddBackward0>)
tensor(0.1515, grad_fn=<AddBackward0>)
tensor(0.2263, grad_fn=<AddBackward0>)
tensor(0.0727, grad_fn=<AddBackward0>)
2048
tensor(7.8275, grad_fn=<AddBackward0>)
tensor(12.3001, grad_fn=<AddBackward0>)
tensor(10.9493, grad_fn=<AddBackward0>)
tensor(9.0408, grad_fn=<AddBackward0>)
2048
tensor(0.3022, grad_fn=<AddBackward0>)
tensor(0.3086, grad_fn=<AddBackward0>)
tensor(0.3618, grad_fn=<AddBackward0>)
tensor(0.2499, grad_fn=<AddBackward0>)
2048
tensor(0.2615, grad_fn=<AddBackward0>)
tensor(0.3377, grad_fn=<AddBackward0>)
tensor(0.2497, grad_fn=<AddBackward0>)
tensor(0.3298, grad_fn=<AddBackward0>)
2048
tensor(0.1834, grad_fn=<AddBackward0>)
tensor(0.2196, grad_fn=<AddBa

2048
tensor(10.2622, grad_fn=<AddBackward0>)
tensor(10.7355, grad_fn=<AddBackward0>)
tensor(10.8788, grad_fn=<AddBackward0>)
tensor(9.8857, grad_fn=<AddBackward0>)
2048
tensor(0.0706, grad_fn=<AddBackward0>)
tensor(0.0869, grad_fn=<AddBackward0>)
tensor(0.0718, grad_fn=<AddBackward0>)
tensor(0.0376, grad_fn=<AddBackward0>)
2048
tensor(0.0730, grad_fn=<AddBackward0>)
tensor(0.0502, grad_fn=<AddBackward0>)
tensor(0.0647, grad_fn=<AddBackward0>)
tensor(0.0620, grad_fn=<AddBackward0>)
2048
tensor(2.0295, grad_fn=<AddBackward0>)
tensor(4.0413, grad_fn=<AddBackward0>)
tensor(1.7708, grad_fn=<AddBackward0>)
tensor(4.4228, grad_fn=<AddBackward0>)
2048
tensor(0.0758, grad_fn=<AddBackward0>)
tensor(0.0166, grad_fn=<AddBackward0>)
tensor(0.0296, grad_fn=<AddBackward0>)
tensor(0.1117, grad_fn=<AddBackward0>)
2048
tensor(0.1735, grad_fn=<AddBackward0>)
tensor(0.1007, grad_fn=<AddBackward0>)
tensor(0.0972, grad_fn=<AddBackward0>)
tensor(0.1117, grad_fn=<AddBackward0>)
2048
tensor(0.0912, grad_fn=<Ad

tensor(11.9302, grad_fn=<AddBackward0>)
tensor(7.7533, grad_fn=<AddBackward0>)
tensor(11.3252, grad_fn=<AddBackward0>)
2048
tensor(32.6305, grad_fn=<AddBackward0>)
tensor(37.4447, grad_fn=<AddBackward0>)
tensor(32.6611, grad_fn=<AddBackward0>)
tensor(35.5773, grad_fn=<AddBackward0>)
2048
tensor(33.2096, grad_fn=<AddBackward0>)
tensor(32.7352, grad_fn=<AddBackward0>)
tensor(27.5702, grad_fn=<AddBackward0>)
tensor(36.5906, grad_fn=<AddBackward0>)
2048
tensor(27.3151, grad_fn=<AddBackward0>)
tensor(37.0020, grad_fn=<AddBackward0>)
tensor(34.4508, grad_fn=<AddBackward0>)
tensor(27.0527, grad_fn=<AddBackward0>)
2048
tensor(23.8567, grad_fn=<AddBackward0>)
tensor(22.7242, grad_fn=<AddBackward0>)
tensor(23.5324, grad_fn=<AddBackward0>)
tensor(21.8429, grad_fn=<AddBackward0>)
2048
tensor(36.1844, grad_fn=<AddBackward0>)
tensor(29.9729, grad_fn=<AddBackward0>)
tensor(31.4542, grad_fn=<AddBackward0>)
tensor(31.7128, grad_fn=<AddBackward0>)
2048
tensor(16.8462, grad_fn=<AddBackward0>)
tensor(19.8

tensor(0.0839, grad_fn=<AddBackward0>)
tensor(0.0498, grad_fn=<AddBackward0>)
tensor(0.1125, grad_fn=<AddBackward0>)
2048
tensor(6.7691, grad_fn=<AddBackward0>)
tensor(8.5676, grad_fn=<AddBackward0>)
tensor(9.6110, grad_fn=<AddBackward0>)
tensor(5.7045, grad_fn=<AddBackward0>)
2048
tensor(0.1525, grad_fn=<AddBackward0>)
tensor(0.0555, grad_fn=<AddBackward0>)
tensor(0.1061, grad_fn=<AddBackward0>)
tensor(0.0801, grad_fn=<AddBackward0>)
2048
tensor(0.0848, grad_fn=<AddBackward0>)
tensor(0.1274, grad_fn=<AddBackward0>)
tensor(0.0939, grad_fn=<AddBackward0>)
tensor(0.0888, grad_fn=<AddBackward0>)
2048
tensor(0.1853, grad_fn=<AddBackward0>)
tensor(0.1890, grad_fn=<AddBackward0>)
tensor(0.1968, grad_fn=<AddBackward0>)
tensor(0.1624, grad_fn=<AddBackward0>)
2048
tensor(0.0500, grad_fn=<AddBackward0>)
tensor(0.0106, grad_fn=<AddBackward0>)
tensor(0.0603, grad_fn=<AddBackward0>)
tensor(-0.0049, grad_fn=<AddBackward0>)
2048
tensor(0.0537, grad_fn=<AddBackward0>)
tensor(0.0650, grad_fn=<AddBackwa

tensor(0.1110, grad_fn=<AddBackward0>)
tensor(0.1726, grad_fn=<AddBackward0>)
tensor(0.0779, grad_fn=<AddBackward0>)
2048
tensor(0.2092, grad_fn=<AddBackward0>)
tensor(0.1715, grad_fn=<AddBackward0>)
tensor(0.1985, grad_fn=<AddBackward0>)
tensor(0.1749, grad_fn=<AddBackward0>)
2048
tensor(0.1410, grad_fn=<AddBackward0>)
tensor(0.2559, grad_fn=<AddBackward0>)
tensor(0.1898, grad_fn=<AddBackward0>)
tensor(0.2119, grad_fn=<AddBackward0>)
2048
tensor(0.0666, grad_fn=<AddBackward0>)
tensor(0.1149, grad_fn=<AddBackward0>)
tensor(0.1048, grad_fn=<AddBackward0>)
tensor(0.0857, grad_fn=<AddBackward0>)
2048
tensor(0.1386, grad_fn=<AddBackward0>)
tensor(0.1368, grad_fn=<AddBackward0>)
tensor(0.1661, grad_fn=<AddBackward0>)
tensor(0.1296, grad_fn=<AddBackward0>)
2048
tensor(0.1235, grad_fn=<AddBackward0>)
tensor(0.1350, grad_fn=<AddBackward0>)
tensor(0.1128, grad_fn=<AddBackward0>)
tensor(0.1517, grad_fn=<AddBackward0>)
2048
tensor(0.0967, grad_fn=<AddBackward0>)
tensor(0.0930, grad_fn=<AddBackwar

# Analysis

In [26]:
if not len(all_returns)%configs.num_episodes_to_average==0:
    all_returns_truncated = np.array(all_returns[:-(len(all_returns)%configs.num_episodes_to_average)])
else:
    all_returns_truncated = all_returns
all_returns_smoothed = np.average(all_returns_truncated.reshape(-1, configs.num_episodes_to_average), axis=1)
print('mean reward:', np.mean(all_returns_smoothed))
print('std reward:', np.std(all_returns_smoothed))
print('max reward:', np.max(all_returns_smoothed))
print('converge mean reward:', np.mean(all_returns_smoothed[-1]))
plt.plot(all_returns_smoothed);


AttributeError: 'list' object has no attribute 'reshape'

## Inference

In [101]:
# create a test env
test_env = make_env_func(configs.gym_id, seed, 0, 'inference', True)()

# use the trained agent to run through the env till it terminates this is an eposide
observation, _ = test_env.reset()
observation = torch.unsqueeze(torch.tensor(observation),dim=0).to(device)
for _ in range(500):
    action, _, _, _ = agent.policy(observation)
    action = action.cpu().item()
    observation, reward, done, _, info = test_env.step(action)
    observation = torch.unsqueeze(torch.tensor(observation),dim=0).to(device)
    if done:
        break
test_env.close()

Video('/content/videos/inference/rl-video-episode-0.mp4', embed=True)

/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /Users/cohlem/Projects/Experimentation/Deep Learning/RLHF/PPO/videos/inference folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [30]:
Video('/content/videos/inference/rl-video-episode-0.mp4', embed=True)

### BreakOut

You can use similar PPO setup to solve much more complex problems. The problem `"gym_id": "BreakoutNoFrameskip-v4"` is left as an exercise for you. The `env` and `agent` definitions are provided here but the rest are left to you. With the correct setup (very similar to what we did in CartPole), you should be able to get to respectable scores above 700. Even perfect score is possible with a tuned hyper-parameters with the same setup. Note that this is a much more complex problem and you will need to increase the `total_timesteps` by at least a factor of 30 to get to good results. This will take several hours.

```python
# extra imports
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
)


# env definition
def make_env_func(gym_id, seed, idx, run_name, capture_video=False):
    def env_fun():
        env = gym.make(gym_id, render_mode='rgb_array')
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            # initiate the video capture if not already initiated
            if idx == 0:
                # wrapper to create the video of the performance
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return env_fun


# Model defnition
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

class Agent(nn.Module):
    def __init__(self, envs):
        super(Agent, self).__init__()
        self.network = nn.Sequential(
            layer_init(nn.Conv2d(4, 32, 8, stride=4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride=1)),
            nn.AdaptiveMaxPool2d(output_size=(1, 1)),
            nn.Flatten(),
            layer_init(nn.Linear(64, 512)),
            nn.ReLU(),
        )
        self.value_head = layer_init(nn.Linear(512, 1), std=1)
        self.policy_head = layer_init(nn.Linear(512, envs.single_action_space.n), std=0.01)

    def value_func(self, x):
        return self.value_head(self.network(x / 255.0))

    def policy(self, x, action=None):
        hidden = self.network(x / 255.0)
        logits = self.policy_head(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.value_head(hidden)
```

## Complete

In [9]:
# create an env with random state
def make_env_func(gym_id, seed, idx, run_name, capture_video=False):
    def env_fun():
        env = gym.make(gym_id, render_mode="rgb_array")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            # initiate the video capture if not already initiated
            if idx == 0:
                # wrapper to create the video of the performance
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return env_fun

In [20]:


# create N envs
envs = []
for i in range(configs.num_trajcts):
    envs.append( make_env_func(configs.gym_id, seed + i, i, run_name) )
envs = gym.vector.SyncVectorEnv(envs)

# start the environment
cur_observation = envs.reset()[0]

class FCBlock(nn.Module):
    """A generic fully connected residual block with good setup"""
    def __init__(self, embd_dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.LayerNorm(embd_dim),
            nn.GELU(),
            nn.Linear(embd_dim, 4*embd_dim),
            nn.GELU(),
            nn.Linear(4*embd_dim, embd_dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return x + self.block(x)



class Agent(nn.Module):
    """an agent that creates actions and estimates values"""
    def __init__(self, env_observation_dim, action_space_dim, embd_dim=64, num_blocks=2):
        super().__init__()
        self.embedding_layer = nn.Linear(env_observation_dim, embd_dim)
        self.shared_layers = nn.Sequential(*[FCBlock(embd_dim=embd_dim) for _ in range(num_blocks)])
        self.value_head = nn.Linear(embd_dim, 1)
        self.policy_head = nn.Linear(embd_dim, action_space_dim)
        # orthogonal initialization with a hi entropy for more exploration at the start
        torch.nn.init.orthogonal_(self.policy_head.weight, 0.01)

    def value_func(self, state):
        hidden = self.shared_layers(self.embedding_layer(state))
        value = self.value_head(hidden)
        return value

    def policy(self, state, action=None):
        hidden = self.shared_layers(self.embedding_layer(state))
        logits = self.policy_head(hidden)
        # PyTorch categorical class helpful for sampling and probability calculations
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.value_head(hidden)

#     "num_trajcts": 32, # N
#     "max_trajects_length": 64, # T

def create_rollout(envs, cur_observation, cur_done, agent):

    observations = torch.zeros((cur_observation.shape[0], configs['max_trajects_length'],envs.single_observation_space.shape[0] ))
    actions = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']) + envs.single_action_space.shape)
    dones =  torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    rewards =  torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    values = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    advantages = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    logprobs = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
     
    
    for t in range(configs['max_trajects_length']):
        # get the policy
        with torch.no_grad():
            action, logprob,_,value = agent.policy(cur_observation)
            
        observations[:,t,:] = cur_observation
        actions[:,t] = action
        dones[:,t] = cur_done
        logprobs[:,t] = logprob
        
        cur_observation, cur_reward, cur_done,_,_ = envs.step(action.cpu().numpy())
        cur_observation = torch.tensor(cur_observation)
        cur_reward = torch.tensor(cur_reward)
        cur_done = torch.tensor(cur_done)
        
        rewards[:,t] = torch.tensor(cur_reward)

        values[:,t] = value.squeeze()
       
    # Advantage is approximated reverse recursively
#     advantage = gae(observations,dones, rewards,values, advantages)
    
    return {
        "cur_observation": cur_observation,
        "observations": observations,
        "actions" : actions,
        "dones" : dones,
        "rewards" : rewards,
        "values" : values,
        "advantages" : advantages,
        "logprobs" : logprobs
    }

agent = Agent(
    env_observation_dim=envs.single_observation_space.shape[0],
    action_space_dim=envs.single_action_space.n
).to(device)

optimizer = optim.Adam(agent.parameters(), lr=configs.learning_rate)


def gae(rewards, values, dones, cur_observation, agent):
    last_advantage = 0
    with torch.no_grad():
        last_value = agent.value_func(cur_observation).reshape(1,-1)
    advantages = torch.zeros_like(values)
    
    for t in reversed(range(configs['max_trajects_length'])):
        mask = 1.0 - dones[:,t]
        last_advantage = mask*last_advantage
        last_value = mask*last_value
        delta = rewards[:,t] + configs['gamma']*last_value - values[:,t]
        advantages[:,t] = delta + configs['gae_lambda']*configs['gamma']*last_advantage
        last_value = values[:,t]
        last_advantage = advantages[:,t]
    
    returns = advantages + values
        
    return advantages, returns


def loss_clip(
    mb_oldlogporb,     # old logprob of mini batch actions collected during the rollout
    mb_newlogprob,     # new logprob of mini batch actions created by the new policy
    mb_advantages      # mini batch of advantages collected during the the rollout
):
    """
    policy loss with clipping to control gradients
    """
    ratio = torch.exp(mb_newlogprob - mb_oldlogporb)
    policy_loss = -mb_advantages * ratio
    # clipped policy gradient loss enforces closeness
    clipped_loss = -mb_advantages * torch.clamp(ratio, 1 - configs.clip_epsilon, 1 + configs.clip_epsilon)
    pessimistic_loss = torch.max(policy_loss, clipped_loss).mean()
    return pessimistic_loss


def loss_vf(
    mb_oldreturns,  # mini batch of old returns collected during the rollout
    mb_newvalues    # minibach of values calculated by the new value function
):
    """
    enforcing the value function to give more accurate estimates of returns
    """
    mb_newvalues = mb_newvalues.view(-1)
    loss = 0.5 * ((mb_newvalues - mb_oldreturns) ** 2).mean()
    return loss


class Storage(Dataset):
    def __init__(self, rollout, advantages, returns, envs):
        # fill in the storage and flatten the parallel trajectories
        self.observations = rollout['observations'].reshape((-1,) + envs.single_observation_space.shape)
        self.logprobs = rollout['logprobs'].reshape(-1)
        self.actions = rollout['actions'].reshape((-1,) + envs.single_action_space.shape).long()
        self.advantages = advantages.reshape(-1)
        self.returns = returns.reshape(-1)

    def __getitem__(self, ix: int):
        item = [
            self.observations[ix],
            self.logprobs[ix],
            self.actions[ix],
            self.advantages[ix],
            self.returns[ix]
        ]
        return item

    def __len__(self) -> int:
        return len(self.observations)
    
import torch
from torch.distributions.categorical import Categorical
# Create the environment
envs = []
for i in range(configs.num_trajcts):
    envs.append( make_env_func(configs.gym_id, seed + i, i, run_name) )
envs = gym.vector.SyncVectorEnv(envs)
cur_observation = torch.tensor(envs.reset()[0])
cur_done = torch.tensor(torch.zeros(configs.num_trajcts))


for i in range(int((configs.total_timesteps/configs.batch_size))):

    frac = 1.0 - (i - 1.0) / (configs.total_timesteps/configs.batch_size)
    optimizer.param_groups[0]["lr"] = frac * configs.learning_rate
    # Phase 1: Create rollout
    rollouts = create_rollout(envs,cur_observation,cur_done, agent)
    advantages,returns = gae(rollouts['rewards'], rollouts['values'], rollouts['dones'], rollouts['cur_observation'], agent)
#     rollout, advantages, returns, envs
    dataset = Storage(rollouts, advantages, returns, envs)
    dataloader = DataLoader(dataset, batch_size=configs.minibatch_size, shuffle=True)
    
    # Phase 2: Update
    for j in range(configs.update_epochs):
        for data in dataloader: # mini_batch
            mb_ob,mb_logprobs, mb_actions, mb_advantages, mb_returns = data

            new_actions, new_logprobs, new_entropy, new_values = agent.policy(mb_ob, mb_actions)
            
#             print(f'old logprobs: {mb_logprobs[0]}, new_logprobs: {new_logprobs[0]}, old advantage: {mb_advantages[0]}, old return: {mb_returns[0]}, new values: {new_values[0]} ')
            
#             print(new_logprobs[0] / mb_logprobs[0]*mb_advantages[0])
            c_loss = loss_clip(mb_logprobs, new_logprobs, mb_advantages)
            vf_loss = loss_vf(mb_returns, new_values)
            entropy = new_entropy.mean()
            
            # maximize the PPO clip loss, minimize the value loss and maximize the entropy(exploration)
            # since pytorch's optimizer are configured to do gradient descent under the hood, i.e W - lr* deltaW
            # we need to multiply the loss the with negative(-) if we need to do gradient ascent
            # we already multiplied it in the clip loss function and we multiply it in entropy.
            
            loss = c_loss + configs.vf_coef*vf_loss - configs.ent_coef*entropy
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), configs.max_grad_norm)
            optimizer.step()
            
            print('----------------------------------------------------------')
            print(f'TOTAL LOSS, {loss}, CLIP loss: {c_loss}, value function loss: {vf_loss}, entropy loss: {entropy}')


/var/folders/pv/2c84c3jx48v8485vx92jq6zc0000gn/T/ipykernel_9366/1773201375.py:185: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cur_done = torch.tensor(torch.zeros(configs.num_trajcts))
/var/folders/pv/2c84c3jx48v8485vx92jq6zc0000gn/T/ipykernel_9366/1773201375.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards[:,t] = torch.tensor(cur_reward)


----------------------------------------------------------
TOTAL LOSS, 10.348729133605957, CLIP loss: -7.379720687866211, value function loss: 35.470760345458984, entropy loss: 0.6930763721466064
----------------------------------------------------------
TOTAL LOSS, 8.243270874023438, CLIP loss: -7.169240951538086, value function loss: 30.838886260986328, entropy loss: 0.6930961012840271
----------------------------------------------------------
TOTAL LOSS, 7.106352806091309, CLIP loss: -7.216591835021973, value function loss: 28.659748077392578, entropy loss: 0.6929196715354919
----------------------------------------------------------
TOTAL LOSS, 6.165403842926025, CLIP loss: -7.365848541259766, value function loss: 27.076356887817383, entropy loss: 0.6925910711288452
----------------------------------------------------------
TOTAL LOSS, 10.36641788482666, CLIP loss: -7.085920810699463, value function loss: 34.91851806640625, entropy loss: 0.6920415163040161
-------------------------

----------------------------------------------------------
TOTAL LOSS, 15.944039344787598, CLIP loss: -0.7599526047706604, value function loss: 33.42106628417969, entropy loss: 0.6541071534156799
----------------------------------------------------------
TOTAL LOSS, 15.573347091674805, CLIP loss: -1.3486592769622803, value function loss: 33.85710144042969, entropy loss: 0.6543787717819214
----------------------------------------------------------
TOTAL LOSS, 15.704325675964355, CLIP loss: -1.150803804397583, value function loss: 33.72331237792969, entropy loss: 0.6526653170585632
----------------------------------------------------------
TOTAL LOSS, 14.707890510559082, CLIP loss: -0.9192712306976318, value function loss: 31.267274856567383, entropy loss: 0.6475383639335632
----------------------------------------------------------
TOTAL LOSS, 17.696914672851562, CLIP loss: -1.1446342468261719, value function loss: 37.696044921875, entropy loss: 0.6473550796508789
----------------------

----------------------------------------------------------
TOTAL LOSS, 17.09221839904785, CLIP loss: -2.036982297897339, value function loss: 38.2701301574707, entropy loss: 0.586458683013916
----------------------------------------------------------
TOTAL LOSS, 20.072446823120117, CLIP loss: -1.3047281503677368, value function loss: 42.76608657836914, entropy loss: 0.5868306159973145
----------------------------------------------------------
TOTAL LOSS, 17.104907989501953, CLIP loss: -1.9703298807144165, value function loss: 38.1622314453125, entropy loss: 0.5877543091773987
----------------------------------------------------------
TOTAL LOSS, 14.065033912658691, CLIP loss: -2.565640449523926, value function loss: 33.27303695678711, entropy loss: 0.5844131708145142
----------------------------------------------------------
TOTAL LOSS, 14.577067375183105, CLIP loss: -2.1921632289886475, value function loss: 33.55002975463867, entropy loss: 0.5784040689468384
--------------------------

----------------------------------------------------------
TOTAL LOSS, 14.473060607910156, CLIP loss: -1.9907119274139404, value function loss: 32.93883514404297, entropy loss: 0.5644832849502563
----------------------------------------------------------
TOTAL LOSS, 13.008715629577637, CLIP loss: -2.196770668029785, value function loss: 30.422138214111328, entropy loss: 0.5583270788192749
----------------------------------------------------------
TOTAL LOSS, 13.38146686553955, CLIP loss: -2.1364383697509766, value function loss: 31.0470027923584, entropy loss: 0.5595700740814209
----------------------------------------------------------
TOTAL LOSS, 15.919726371765137, CLIP loss: -1.0064388513565063, value function loss: 33.86322021484375, entropy loss: 0.5444563031196594
----------------------------------------------------------
TOTAL LOSS, 17.620643615722656, CLIP loss: -0.7727962136268616, value function loss: 36.79756164550781, entropy loss: 0.5340909957885742
----------------------

...

----------------------------------------------------------
TOTAL LOSS, 9.945066452026367, CLIP loss: 0.650600016117096, value function loss: 18.599903106689453, entropy loss: 0.5485504865646362
----------------------------------------------------------
TOTAL LOSS, 0.10063993185758591, CLIP loss: -0.09575165063142776, value function loss: 0.4035267233848572, entropy loss: 0.5371780395507812
----------------------------------------------------------
TOTAL LOSS, 0.15937146544456482, CLIP loss: -0.057165637612342834, value function loss: 0.4440588057041168, entropy loss: 0.549229621887207
----------------------------------------------------------
TOTAL LOSS, 0.13728155195713043, CLIP loss: -0.0728650614619255, value function loss: 0.43115997314453125, entropy loss: 0.5433366298675537
----------------------------------------------------------
TOTAL LOSS, 0.16389863193035126, CLIP loss: -0.08908496797084808, value function loss: 0.5166406631469727, entropy loss: 0.5336726903915405
----------

----------------------------------------------------------
TOTAL LOSS, 0.1761694699525833, CLIP loss: -0.03005749173462391, value function loss: 0.42310455441474915, entropy loss: 0.5325312614440918
----------------------------------------------------------
TOTAL LOSS, 0.20497910678386688, CLIP loss: 0.0046586692333221436, value function loss: 0.41146421432495117, entropy loss: 0.5411669015884399
----------------------------------------------------------
TOTAL LOSS, 0.15268145501613617, CLIP loss: -0.037881240248680115, value function loss: 0.3919435143470764, entropy loss: 0.5409059524536133
----------------------------------------------------------
TOTAL LOSS, 0.18079273402690887, CLIP loss: 0.0101285669952631, value function loss: 0.35199233889579773, entropy loss: 0.5332001447677612
----------------------------------------------------------
TOTAL LOSS, 0.17114481329917908, CLIP loss: -0.037939008325338364, value function loss: 0.4291887581348419, entropy loss: 0.5510554909706116
--

----------------------------------------------------------
TOTAL LOSS, 0.16579407453536987, CLIP loss: -0.05761966109275818, value function loss: 0.45766568183898926, entropy loss: 0.54190993309021
----------------------------------------------------------
TOTAL LOSS, 0.23363091051578522, CLIP loss: 0.0608680434525013, value function loss: 0.3563547134399414, entropy loss: 0.5414479970932007
----------------------------------------------------------
TOTAL LOSS, 0.2666631042957306, CLIP loss: 0.05133798345923424, value function loss: 0.441510945558548, entropy loss: 0.5430347919464111
----------------------------------------------------------
TOTAL LOSS, 0.2894027531147003, CLIP loss: 0.08983919024467468, value function loss: 0.4098123610019684, entropy loss: 0.5342628955841064
----------------------------------------------------------
TOTAL LOSS, 0.18460537493228912, CLIP loss: 0.021160468459129333, value function loss: 0.33772972226142883, entropy loss: 0.5419955253601074
------------

----------------------------------------------------------
TOTAL LOSS, 0.22180813550949097, CLIP loss: 0.019760170951485634, value function loss: 0.41468098759651184, entropy loss: 0.5292534232139587
----------------------------------------------------------
TOTAL LOSS, 0.18204975128173828, CLIP loss: -0.03261308744549751, value function loss: 0.44014379382133484, entropy loss: 0.5409061312675476
----------------------------------------------------------
TOTAL LOSS, 0.16234055161476135, CLIP loss: -0.024120137095451355, value function loss: 0.38379746675491333, entropy loss: 0.543804407119751
----------------------------------------------------------
TOTAL LOSS, 0.1867704689502716, CLIP loss: -0.0028853067196905613, value function loss: 0.3900943100452423, entropy loss: 0.5391373634338379
----------------------------------------------------------
TOTAL LOSS, 0.14214996993541718, CLIP loss: -0.04056426137685776, value function loss: 0.3763887882232666, entropy loss: 0.5480149984359741
-

----------------------------------------------------------
TOTAL LOSS, 0.10775760561227798, CLIP loss: -0.08655533939599991, value function loss: 0.3991553783416748, entropy loss: 0.52647465467453
----------------------------------------------------------
TOTAL LOSS, 0.1484443098306656, CLIP loss: -0.049621663987636566, value function loss: 0.4069461226463318, entropy loss: 0.5407078266143799
----------------------------------------------------------
TOTAL LOSS, 0.21052229404449463, CLIP loss: -0.002876847982406616, value function loss: 0.4375477731227875, entropy loss: 0.5374749898910522
----------------------------------------------------------
TOTAL LOSS, 0.06140885502099991, CLIP loss: -0.1331312656402588, value function loss: 0.3997878134250641, entropy loss: 0.5353787541389465
----------------------------------------------------------
TOTAL LOSS, 0.21687918901443481, CLIP loss: 0.015298177488148212, value function loss: 0.41405168175697327, entropy loss: 0.5444822311401367
------

In [8]:
# create an env with random state
def make_env_func(gym_id, seed, idx, run_name, capture_video=False):
    def env_fun():
        env = gym.make(gym_id, render_mode="rgb_array")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            # initiate the video capture if not already initiated
            if idx == 0:
                # wrapper to create the video of the performance
                env = gym.wrapper
                s.RecordVideo(env, f"videos/{run_name}")
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return env_fun

In [9]:
# create N envs
envs = []
for i in range(configs.num_trajcts):
    envs.append( make_env_func(configs.gym_id, seed + i, i, run_name) )
envs = gym.vector.SyncVectorEnv(envs)
envs

SyncVectorEnv(num_envs=32)

In [10]:
# start the environment
cur_observation = envs.reset()[0]

In [11]:
class FCBlock(nn.Module):
    """A generic fully connected residual block with good setup"""
    def __init__(self, embd_dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.LayerNorm(embd_dim),
            nn.GELU(),
            nn.Linear(embd_dim, 4*embd_dim),
            nn.GELU(),
            nn.Linear(4*embd_dim, embd_dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return x + self.block(x)



class Agent(nn.Module):
    """an agent that creates actions and estimates values"""
    def __init__(self, env_observation_dim, action_space_dim, embd_dim=64, num_blocks=2):
        super().__init__()
        self.embedding_layer = nn.Linear(env_observation_dim, embd_dim)
        self.shared_layers = nn.Sequential(*[FCBlock(embd_dim=embd_dim) for _ in range(num_blocks)])
        self.value_head = nn.Linear(embd_dim, 1)
        self.policy_head = nn.Linear(embd_dim, action_space_dim)
        # orthogonal initialization with a hi entropy for more exploration at the start
        torch.nn.init.orthogonal_(self.policy_head.weight, 0.01)

    def value_func(self, state):
        hidden = self.shared_layers(self.embedding_layer(state))
        value = self.value_head(hidden)
        return value

    def policy(self, state, action=None):
        hidden = self.shared_layers(self.embedding_layer(state))
        logits = self.policy_head(hidden)
        # PyTorch categorical class helpful for sampling and probability calculations
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.value_head(hidden)


In [12]:
#     "num_trajcts": 32, # N
#     "max_trajects_length": 64, # T

def create_rollout(envs, cur_observation, cur_done, agent):

    observations = torch.zeros((cur_observation.shape[0], configs['max_trajects_length'],envs.single_observation_space.shape[0] ))
    actions = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']) + envs.single_action_space.shape)
    dones =  torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    rewards =  torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    values = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    advantages = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
    logprobs = torch.zeros((cur_observation.shape[0], configs['max_trajects_length']))
     
    
    for t in range(configs['max_trajects_length']):
        # get the policy
        with torch.no_grad():
            action, logprobb,_,value = agent.policy(cur_observation)
            
        observations[:,t,:] = cur_observation
        actions[:,t] = action
        dones[:,t] = cur_done
        
        cur_observation, cur_reward, cur_done,_,_ = envs.step(action.cpu().numpy())
        cur_observation = torch.tensor(cur_observation)
        cur_reward = torch.tensor(cur_reward)
        cur_done = torch.tensor(cur_done)
        
        rewards[:,t] = torch.tensor(cur_reward)

        values[:,t] = value.squeeze()
       
    # Advantage is approximated reverse recursively
#     advantage = gae(observations,dones, rewards,values, advantages)
    
    return {
        "cur_observation": cur_observation,
        "observations": observations,
        "actions" : actions,
        "dones" : dones,
        "rewards" : rewards,
        "values" : values,
        "advantages" : advantages,
        "logprobs" : logprobs
    }

In [13]:
agent = Agent(
    env_observation_dim=envs.single_observation_space.shape[0],
    action_space_dim=envs.single_action_space.n
).to(device)

optimizer = optim.Adam(agent.parameters(), lr=configs.learning_rate)

In [14]:
rollouts = create_rollout(envs, torch.tensor(cur_observation), torch.tensor(cur_done), agent)


NameError: name 'cur_done' is not defined

In [15]:
cur_observation = rollouts['cur_observation']
values = rollouts['values']
dones = rollouts['dones']
rewards = rollouts['rewards']


NameError: name 'rollouts' is not defined

In [16]:
def gae(rewards, values, dones, cur_observation, agent):
    last_advantage = 0
    with torch.no_grad():
        last_value = agent.value_func(cur_observation).reshape(1,-1)
    advantages = torch.zeros_like(values)
    
    for t in reversed(range(configs['max_trajects_length'])):
        mask = 1.0 - dones[:,t]
        last_advantage = mask*last_advantage
        last_value = mask*last_value
        delta = rewards[:,t] + configs['gamma']*last_value - values[:,t]
        advantages[:,t] = delta + configs['gae_lambda']*configs['gamma']*last_advantage
        last_value = values[:,t]
        last_advantage = advantages[:,t]
    
    returns = advantages + values
        
    return advantages, returns

In [17]:
advantages,returns = gae(rewards, values, dones, cur_observation, agent)

NameError: name 'rewards' is not defined

In [18]:
def gae(rewards, values, dones, cur_observation, agent):
    last_advantage = 0
    with torch.no_grad():
        last_value = agent.value_func(cur_observation).reshape(1,-1)
    advantages = torch.zeros_like(values)
    
    for t in reversed(range(configs['max_trajects_length'])):
        mask = 1.0 - dones[:,t]
        last_advantage = mask*last_advantage
        last_value = mask*last_value
        delta = rewards[:,t] + configs['gamma']*last_value - values[:,t]
        advantages[:,t] = delta + configs['gae_lambda']*configs['gamma']*last_advantage
        last_value = values[:,t]
        last_advantage = advantages[:,t]
    
    returns = advantages + values
        
    return advantages, returns


def loss_clip(
    mb_oldlogporb,     # old logprob of mini batch actions collected during the rollout
    mb_newlogprob,     # new logprob of mini batch actions created by the new policy
    mb_advantages      # mini batch of advantages collected during the the rollout
):
    """
    policy loss with clipping to control gradients
    """
    ratio = torch.exp(mb_newlogprob - mb_oldlogporb)
    policy_loss = -mb_advantages * ratio
    # clipped policy gradient loss enforces closeness
    clipped_loss = -mb_advantages * torch.clamp(ratio, 1 - configs.clip_epsilon, 1 + configs.clip_epsilon)
    pessimistic_loss = torch.max(policy_loss, clipped_loss).mean()
    return pessimistic_loss


def loss_vf(
    mb_oldreturns,  # mini batch of old returns collected during the rollout
    mb_newvalues    # minibach of values calculated by the new value function
):
    """
    enforcing the value function to give more accurate estimates of returns
    """
    mb_newvalues = mb_newvalues.view(-1)
    loss = 0.5 * ((mb_newvalues - mb_oldreturns) ** 2).mean()
    return loss


class Storage(Dataset):
    def __init__(self, rollout, advantages, returns, envs):
        # fill in the storage and flatten the parallel trajectories
        self.observations = rollout['observations'].reshape((-1,) + envs.single_observation_space.shape)
        self.logprobs = rollout['logprobs'].reshape(-1)
        self.actions = rollout['actions'].reshape((-1,) + envs.single_action_space.shape).long()
        self.advantages = advantages.reshape(-1)
        self.returns = returns.reshape(-1)

    def __getitem__(self, ix: int):
        item = [
            self.observations[ix],
            self.logprobs[ix],
            self.actions[ix],
            self.advantages[ix],
            self.returns[ix]
        ]
        return item

    def __len__(self) -> int:
        return len(self.observations)
    
import torch
from torch.distributions.categorical import Categorical
# Create the environment
envs = []
for i in range(configs.num_trajcts):
    envs.append( make_env_func(configs.gym_id, seed + i, i, run_name) )
envs = gym.vector.SyncVectorEnv(envs)
cur_observation = torch.tensor(envs.reset()[0])
cur_done = torch.tensor(torch.zeros(configs.num_trajcts))


for i in range(200):

    frac = 1.0 - (i - 1.0) / (configs.total_timesteps/configs.batch_size)
    optimizer.param_groups[0]["lr"] = frac * configs.learning_rate
    # Phase 1: Create rollout
    rollouts = create_rollout(envs,cur_observation,cur_done, agent)
    advantages,returns = gae(rollouts['rewards'], rollouts['values'], rollouts['dones'], rollouts['cur_observation'], agent)
#     rollout, advantages, returns, envs
    dataset = Storage(rollouts, advantages, returns, envs)
    dataloader = DataLoader(dataset, batch_size=configs.minibatch_size, shuffle=True)
    
    # Phase 2: Update
    for j in range(configs.update_epochs):
        for data in dataloader: # mini_batch
            mb_ob,mb_logprobs, mb_actions, mb_advantages, mb_returns = data
            
            new_actions, new_logprobs, new_entropy, new_values = agent.policy(mb_ob, mb_logprobs)
            
            c_loss = loss_clip(mb_logprobs, new_logprobs, mb_advantages)
            vf_loss = loss_vf(mb_returns, new_values)
            entropy = new_entropy.mean()
            
            # maximize the PPO clip loss, minimize the value loss and maximize the entropy(exploration)
            # since pytorch's optimizer are configured to do gradient descent under the hood, i.e W - lr* deltaW
            # we need to multiply the loss the with negative(-) if we need to do gradient ascent
            # we already multiplied it in the clip loss function and we multiply it in entropy.
            
            loss = c_loss + configs.vf_coef*vf_loss - configs.ent_coef*entropy
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), configs.max_grad_norm)
            optimizer.step()
            
            print(loss)
        

In [19]:
configs

{'exp_name': 'cartpole', 'gym_id': 'CartPole-v1', 'learning_rate': 0.001, 'total_timesteps': 1000000, 'max_grad_norm': 0.5, 'num_trajcts': 32, 'max_trajects_length': 64, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 2, 'update_epochs': 2, 'clip_epsilon': 0.2, 'ent_coef': 0.01, 'vf_coef': 0.5, 'num_returns_to_average': 3, 'num_episodes_to_average': 23, 'batch_size': 2048, 'minibatch_size': 1024}

In [20]:
def loss_clip(
    mb_oldlogporb,     # old logprob of mini batch actions collected during the rollout
    mb_newlogprob,     # new logprob of mini batch actions created by the new policy
    mb_advantages      # mini batch of advantages collected during the the rollout
):
    """
    policy loss with clipping to control gradients
    """
    ratio = torch.exp(mb_newlogprob - mb_oldlogporb)
    policy_loss = -mb_advantages * ratio
    # clipped policy gradient loss enforces closeness
    clipped_loss = -mb_advantages * torch.clamp(ratio, 1 - configs.clip_epsilon, 1 + configs.clip_epsilon)
    pessimistic_loss = torch.max(policy_loss, clipped_loss).mean()
    return pessimistic_loss


def loss_vf(
    mb_oldreturns,  # mini batch of old returns collected during the rollout
    mb_newvalues    # minibach of values calculated by the new value function
):
    """
    enforcing the value function to give more accurate estimates of returns
    """
    mb_newvalues = mb_newvalues.view(-1)
    loss = 0.5 * ((mb_newvalues - mb_oldreturns) ** 2).mean()
    return loss

In [21]:
configs

{'exp_name': 'cartpole', 'gym_id': 'CartPole-v1', 'learning_rate': 0.001, 'total_timesteps': 1000000, 'max_grad_norm': 0.5, 'num_trajcts': 32, 'max_trajects_length': 64, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 2, 'update_epochs': 2, 'clip_epsilon': 0.2, 'ent_coef': 0.01, 'vf_coef': 0.5, 'num_returns_to_average': 3, 'num_episodes_to_average': 23, 'batch_size': 2048, 'minibatch_size': 1024}

In [1]:
import torch
from torch.distributions.categorical import Categorical
# Create the environment
envs = []
for i in range(configs.num_trajcts):
    envs.append( make_env_func(configs.gym_id, seed + i, i, run_name) )
envs = gym.vector.SyncVectorEnv(envs)
cur_observation = torch.tensor(envs.reset()[0])
cur_done = torch.tensor(torch.zeros(configs.num_trajcts))


for i in range(200):

    frac = 1.0 - (i - 1.0) / (configs.total_timesteps/configs.batch_size)
    optimizer.param_groups[0]["lr"] = frac * configs.learning_rate
    # Phase 1: Create rollout
    rollouts = create_rollout(envs,cur_observation,cur_done, agent)
    advantages,returns = gae(rollouts['rewards'], rollouts['values'], rollouts['dones'], rollouts['cur_observation'], agent)
#     rollout, advantages, returns, envs
    dataset = Storage(rollouts, advantages, returns, envs)
    dataloader = DataLoader(dataset, batch_size=configs.minibatch_size, shuffle=True)
    
    # Phase 2: Update
    for j in range(configs.update_epochs):
        for data in dataloader: # mini_batch
            mb_ob,mb_logprobs, mb_actions, mb_advantages, mb_returns = data
            
            new_actions, new_logprobs, new_entropy, new_values = agent.policy(mb_ob, mb_logprobs)
            
            print(mb_logprobs[0], new_logprobs[0], mb_advantage[0])
            
            print(new_logprobs[0] / mb_logprobs[0]*mb_advanrage[0])
            c_loss = loss_clip(mb_logprobs, new_logprobs, mb_advantages)
            vf_loss = loss_vf(mb_returns, new_values)
            entropy = new_entropy.mean()
            
            # maximize the PPO clip loss, minimize the value loss and maximize the entropy(exploration)
            # since pytorch's optimizer are configured to do gradient descent under the hood, i.e W - lr* deltaW
            # we need to multiply the loss the with negative(-) if we need to do gradient ascent
            # we already multiplied it in the clip loss function and we multiply it in entropy.
            
            loss = c_loss + configs.vf_coef*vf_loss - configs.ent_coef*entropy
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), configs.max_grad_norm)
            optimizer.step()
            
#             print(loss)
            
            break
        break
    break
        

NameError: name 'configs' is not defined

In [32]:
# create a test env
test_env = make_env_func(configs.gym_id, seed, 0, 'inference', True)()

# use the trained agent to run through the env till it terminates this is an eposide
observation, _ = test_env.reset()
observation = torch.unsqueeze(torch.tensor(observation),dim=0).to(device)
for _ in range(500):
    action, _, _, _ = agent.policy(observation)
    action = action.cpu().item()
    observation, reward, done, _, info = test_env.step(action)
    observation = torch.unsqueeze(torch.tensor(observation),dim=0).to(device)
    if done:
        break
test_env.close()

Video('/content/videos/inference/rl-video-episode-0.mp4', embed=True)

/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /Users/cohlem/Projects/Experimentation/Deep Learning/RLHF/PPO/videos/inference folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [393]:
configs

{'exp_name': 'cartpole', 'gym_id': 'CartPole-v1', 'learning_rate': 0.001, 'total_timesteps': 1000000, 'max_grad_norm': 0.5, 'num_trajcts': 32, 'max_trajects_length': 64, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 2, 'update_epochs': 2, 'clip_epsilon': 0.2, 'ent_coef': 0.01, 'vf_coef': 0.5, 'num_returns_to_average': 3, 'num_episodes_to_average': 23, 'batch_size': 2048, 'minibatch_size': 1024}

In [22]:
sample = torch.randn((2,3))

In [23]:
sample

tensor([[-0.6010, -1.7952, -0.5128],
        [-0.2789,  0.2727, -1.3774]])

In [25]:
sample.mean(dim=1)

tensor([-0.9696, -0.4612])

In [26]:
import torch

# Example: One embedding (1D tensor) and a list of other embeddings (2D tensor)
one_embedding = torch.tensor([1.0, 2.0, 3.0])
list_of_embeddings = torch.tensor([[4.0, 5.0, 6.0],
                                    [7.0, 8.0, 9.0],
                                    [1.0, 0.0, 0.0]])

# Normalize the one embedding
one_embedding_norm = one_embedding / one_embedding.norm()

# Normalize the list of embeddings
list_of_embeddings_norm = list_of_embeddings / list_of_embeddings.norm(dim=1, keepdim=True)

# Compute cosine similarities (dot product of normalized vectors)
cos_similarities = torch.mm(list_of_embeddings_norm, one_embedding_norm.unsqueeze(1)).squeeze()

# Print cosine similarities
print(cos_similarities)


tensor([0.9746, 0.9594, 0.2673])


In [27]:
list_of_embeddings.norm(dim=1, keepdim=True)

tensor([[ 8.7750],
        [13.9284],
        [ 1.0000]])

In [29]:
cos_similarities.argmax()

tensor(0)

In [33]:
# Number of top scores to retrieve (k)
k = 5

# Use torch.topk to get the top-k values and their indices
topk_values, topk_indices = torch.topk(cos_similarities, k)

RuntimeError: selected index k out of range

tensor([0.9746, 0.9594])

In [32]:
topk_indices

tensor([0, 1])

In [42]:
ls = [*list_of_embeddings[0], torch.tensor([0])]

In [44]:
torch.tensor(ls)

tensor([4., 5., 6., 0.])

In [49]:
one = torch.randn(2,3)
print(one)
new_one = one.view(-1)
new_one


tensor([[-0.0150, -0.1535, -0.1722],
        [ 0.9797, -1.4121, -1.5100]])


tensor([-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100])

In [50]:
two = torch.randn(2,3)
print(two)
new_two = two.view(-1)

new_two

tensor([[-0.9105, -1.8257,  1.2258],
        [-0.3211, -0.8037, -1.5332]])


tensor([-0.9105, -1.8257,  1.2258, -0.3211, -0.8037, -1.5332])

In [51]:
torch.cat((new_one, new_two), dim=-1)

tensor([-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, -0.9105, -1.8257,
         1.2258, -0.3211, -0.8037, -1.5332])

In [52]:
import torch

# Original tensor
tensor = torch.tensor([-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, 
                       -0.9105, -1.8257, 1.2258, -0.3211, -0.8037, -1.5332])

# Duplicate across 5 rows
duplicated_tensor = tensor.repeat(5, 1)

# Print the result
print(duplicated_tensor)


tensor([[-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, -0.9105, -1.8257,
          1.2258, -0.3211, -0.8037, -1.5332],
        [-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, -0.9105, -1.8257,
          1.2258, -0.3211, -0.8037, -1.5332],
        [-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, -0.9105, -1.8257,
          1.2258, -0.3211, -0.8037, -1.5332],
        [-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, -0.9105, -1.8257,
          1.2258, -0.3211, -0.8037, -1.5332],
        [-0.0150, -0.1535, -0.1722,  0.9797, -1.4121, -1.5100, -0.9105, -1.8257,
          1.2258, -0.3211, -0.8037, -1.5332]])
